# Sistemas de Recomendación

<div align="center"><a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/MachineLearning/10_Recomendacion/sistemas_recomendacion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg'/> </a> <br> Recordá abrir en una nueva pestaña </div>

In [1]:
import pandas as pd
import numpy as np

## Similitud coseno

$$sim(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}$$

¿Cómo calcularla en Python?

Supongamos que tenemos la siguiente matriz:

|  	| Libro A 	| Libro B 	| Libro C 	|
|-------	|---------	|---------	|---------	|
| Juan 	| 5 	| 4 	| 4 	|
| Diego 	| 4 	| 5 	| 5 	|


Podemos calcular la similitud coseno empleando sklearn:

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
Juan = [5,4,4]
Diego = [4,5,5]
cosine_similarity([Juan, Diego])

array([[1.        , 0.97823198],
       [0.97823198, 1.        ]])

También podemos calcular la similitud a mano:

In [3]:
(5*4 + 4*5 + 4*5)/(np.sqrt(5**2+4**2+4**2)*np.sqrt(4**2+5**2+5**2))

0.9782319760890369

O empleando Numpy

Calcular la similitud coseno usando numpy (con np.dot y np.linalg.norm)

Ahora bien, cuando tenemos una matriz user-item de la vida real, tenemos muchos casos faltantes. En esta situación, no podremos calcular la similitud coseno tan fácilmente... 

In [4]:
user_item = np.array([[5, np.nan, 4],[4,3,5],[4,5,5],[np.nan, 5, np.nan], [np.nan, 5, 3]])
user_item

array([[ 5., nan,  4.],
       [ 4.,  3.,  5.],
       [ 4.,  5.,  5.],
       [nan,  5., nan],
       [nan,  5.,  3.]])

## Surprise

En esta notebook vamos a emplear la librería surprise. Esta es una librería que se basa en la API de scikit-learn y permite implementar varios algoritmos básicos de recomendación.

Comencemos cargando un dataset clásico en sistemas de recomendación: MovieLens (https://movielens.org/). Esta es una página de recomendación de películas que abrió información histórica. 

In [5]:
!pip install surprise
# Bajamos el dataset. En windows pueden descargarlo entrando al link manualmente
!wget https://files.grouplens.org/datasets/movielens/ml-100k/u.data .

     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619416 sha256=d302574156fc55e868203a66a2ea44148c537e35da283718662d6f804d287848
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise
--2021-09-17 20:35:12--  https://files.grouplens.org/datasets/movielens/ml-100k/u.data
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1979173 (1.9M)
Saving to: ‘u.data’

u.data              100%[===================>]   1.89M  --.-KB/s    in 0.1s    

2021-09-17 20:35:12 (14.3 MB/s) - ‘u.data’ saved [1979173/1979173]

--2021-09-17 20:35:12--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host addr

In [6]:
import pandas as pd

In [7]:
mlens = pd.read_csv("u.data",sep="\t",header=None)
mlens.columns = ["user_id","item_id","rating","timestamp"]

In [8]:
mlens = mlens.drop("timestamp", axis=1)

El paquete surprise no recibe directamente un objeto DataFrame sino que tiene para parsear y leer un conjunto de datos debe hacerlo a través de dos nuevos objetos: Reader y Dataset. En Reader debemos especificar el valor mínimo y el valor máximo de los ratings y Dataset nos permite leer datos desde distintas fuentes.

In [9]:
from surprise import Dataset, Reader
reader = Reader(rating_scale=(mlens["rating"].min(),mlens["rating"].max()))

In [10]:
dataset = Dataset.load_from_df(mlens,reader)

In [11]:
dataset

Ahora cargue SVD y GridSearchCV, ambos de surprise.  
Nota: GridSearchCV no está en surprise.GridSearchCV, surprise.GridSearch está deprecado.

In [12]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

Genere una grilla de parámetros donde se prueben distintas combinaciones de:  
  - epochs: es la cantidad de pasadas sobre el dataset que hará el algoritmo empleando descenso por el gradiente  
  - biased: usar parámetros de sesgo o no  
  - lr_all: learning rate para todos los parámetros  
  - reg_all: término de regularización para todos los parámetros (lambda)  

In [13]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

Emplee GridSearchCV, SVD y el diccionario con los parámetros para probar, y entrene un modelo. Note que a GridSearchCV necesita pasarle un modelo sin instanciar. Además, setee el parámetro refit a True y con measures = ["rmse","fcp"]

Imprima el rmse y el fcp, y la mejor combinación de parámetros

Guarde el modelo con mayor fcp y prediga el rating para el user id 196 e item id 242

Pruebe empleando otros modelos como SVDpp, NMF, KNNWithZScore e intente superar el valor obtenido

## Recomendación basada en el contenido

En este ejemplo vamos a tomar un corpus de textos de autores latinoamericanos para sugerir uno similar a uno dado. Para esto construiremos una matriz TFIDF, de frecuencias normalizadas de términos por documento, y usaremos la similitud coseno para medir distancias entre los distintos textos.

In [14]:
!git clone https://github.com/karen-pal/borges

Cloning into 'borges'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 211 (delta 89), reused 171 (delta 49), pack-reused 0
Receiving objects: 100% (211/211), 2.21 MiB | 11.61 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [15]:
import pickle
from pathlib import Path
import pandas as pd

df = pd.DataFrame()
# usando el asterisco de "wildcard" traemos todos los archivos en formato pickle
pkls = Path('.').glob('./borges/datasets/*texts.pkl')

# leemos todos los pickles y concatenarlos en un DataFrame
for pkl in pkls:
    with open(pkl, 'rb') as inp:
        df_ = pickle.load(inp)
    df = pd.concat([df, df_])

df.shape

(719, 3)

In [16]:
df.sample(2)

,link,text_metadata,text
7,https://ciudadseva.com/texto/la-primera-nevada/,"{'title': 'La primera nevada', 'metadata': '[C...",Los objetos que me dejó Torroba se fueron inco...
17,https://ciudadseva.com/texto/el-prodigioso-mil...,"{'title': 'El prodigioso miligramo', 'metadata...",Una hormiga censurada por la sutileza de sus c...


In [17]:
# separamos de la metadata el título y autor en sus propias columnas
df['title'] = df['text_metadata'].apply(lambda x: x['title'])
df['author'] = df['text_metadata'].apply(lambda x: x['author'])

In [18]:
# vemos los autores disponibloes
df['author'].value_counts()

Jorge Luis Borges             60
Julio Cortázar                55
Baldomero Lillo               50
Augusto Monterroso            45
Juan José Arreola             45
Alfonso Reyes                 37
Enrique Anderson Imbert       36
Mario Benedetti               33
Julio Ramón Ribeyro           27
Roberto Arlt                  25
Clarice Lispector             25
Julio Torri                   23
Felisberto Hernández          15
Luis Vidales                  14
Adolfo Bioy Casares           13
Rubén Darío                   13
Álvaro Mutis                  11
Edmundo Valadés               10
Juan Rulfo                    10
Juan Rodolfo Wilcock          10
Elena Garro                    9
Manuel A. Alonso               9
Salarrué                       9
Juan Bosch                     8
Alejo Carpentier               8
Eduardo Gudiño Kieffer         8
Virgilio Díaz Grullón          7
Andrés Rivera                  7
Silvina Ocampo                 7
Rodolfo Walsh                  6
Ricardo Gü

In [19]:
# quitamos duplicados y reiniciamos el índice
df = df.drop_duplicates(subset=[c for c in df.columns if c != 'text_metadata'])
df = df.reset_index(drop=True)
df.shape

(693, 5)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pprint import pprint

Vamos a calcular las matrices de ocurrencias de términos usando sklearn.

Ámbas clases primero construyen el vocabulario total, y luego:  
- **CountVectorizer** nos devuelve la frecuencia absoluta de cada término por cada documento.
- [**TF-IDF**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf): calcula la frecuencia de cada término por documento, y normaliza por el total de documentos donde el término aparece.

$${tf} (t,d)={\frac {f_{t,d}}{\sum _{t'\in d}{f_{t',d}}}}$$

$$
idf( t, D ) = log \frac{ \text{| } D \text{ |} }{ 1 + \text{| } \{ d \in D : t \in d \} \text{ |} }
$$ 


$$ tfidf( t, d, D ) = tf( t, d ) \times idf( t, D ) 
$$


In [21]:
# Instanciamos el CV
vectorizer = CountVectorizer()

doc1 = 'la matriz de frecuencias por palabras otorga información del contenido de un documento'
doc2 = 'las palabras que aparecen en un documento se relaciona con su tema'
# Definimos una lista con todos los strings
data_corpus = [doc1, doc2]

# Fiteamos el CV y transformamos los datos
X = vectorizer.fit_transform(data_corpus) 

# Pasamos de sparse matrix a array usando .toarray()

print(X.toarray())
# Usando el metodo .get_feature_names() del CV podemos acceder al indice de palabras
print(vectorizer.get_feature_names())

[[0 0 1 2 1 1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1]
 [1 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1]]
['aparecen', 'con', 'contenido', 'de', 'del', 'documento', 'en', 'frecuencias', 'información', 'la', 'las', 'matriz', 'otorga', 'palabras', 'por', 'que', 'relaciona', 'se', 'su', 'tema', 'un']


In [22]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = list(stopwords.words('spanish'))
# eliminamos las "stop words", palabras comunes no informativas
tf = TfidfVectorizer(stop_words=stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
# calculamos los features para cada ítem (texto)
tfidf_matrix = tf.fit_transform(df['text'])

In [24]:
# calculamos las similitudes entre todos los documentos
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
n = 6

# diccionario creado para guardar el resultado en un formato (autor - titulo : puntaje, titulo, autor)
results = {} 
for idx, row in df.iterrows():
    # guardamos los indices similares basados en la similitud coseno. Los ordenamos en modo ascendente, siendo 0 nada de similitud y 1 total
    similar_indices = cosine_similarities[idx].argsort()[:-n-2:-1] 
    # guardamos los N más cercanos
    similar_items = [(f"{df['author'][i]} - {df['title'][i]}", round(cosine_similarities[idx][i], 3)) for i in similar_indices]
    results[f"{row['author']} - {row['title']}"] = similar_items[1:]

In [25]:
pprint(results['Jorge Luis Borges - El Aleph'])

[('Jorge Luis Borges - La escritura del dios', 0.144),
 ('Jorge Luis Borges - El inmortal', 0.135),
 ('Jorge Luis Borges - Utopía de un hombre que está cansado', 0.125),
 ('Felisberto Hernández - El acomodador', 0.122),
 ('Clarice Lispector - La búsqueda de la dignidad', 0.121),
 ('Jorge Luis Borges - Funes el memorioso', 0.11)]


In [26]:
def recomendar(autor, titulo):
    pprint(results[f"{autor} - {titulo}"])

In [27]:
recomendar('Julio Cortázar', 'Axolotl')

[('Felisberto Hernández - El acomodador', 0.134),
 ('Felisberto Hernández - El cocodrilo', 0.101),
 ('Felisberto Hernández - Menos Julia', 0.089),
 ('Julio Cortázar - Después del almuerzo', 0.088),
 ('Julio Cortázar - La noche boca arriba', 0.086),
 ('Julio Cortázar - La señorita Cora', 0.086)]
